### Uso de ScikitLearn y prueba del modelo con distintos algoritmos de clasificación para hacer el análisis de sentimiento de tweets.

In [1]:
# imports y configuraciones necesarias
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SparkSession

from pymongo import MongoClient

import pandas as pd
import matplotlib.pyplot as plt

import re
import string
import unicodedata

import nltk
from nltk import TweetTokenizer
from nltk.corpus import stopwords

from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier


conf = (SparkSession\
          .builder\
          .appName("twitter")\
          .master("spark://MacBook-Pro-de-Jose.local:7077")\
          .config("spark.io.compression.codec", "snappy")\
          .getOrCreate())  

sc = SparkContext.getOrCreate(conf=conf)

#### Creación de funciones necesarias, intentando modular y limpiar el notebook de código repetitivo.

In [2]:
# Función de carga de datos desde MongoDB de los datos almacenados desde Apache Nifi, tanto en inglés como español
def carga_datos_mongo():
    # conectamos con las tablas de Mongo desde donde cargamos los datos
    df_mongo_english = spark.read.format("com.mongodb.spark.sql.DefaultSource")\
        .option("spark.mongodb.input.uri", "mongodb://localhost:27017/tfm_twitter.tweets_english").load()
    df_mongo_spanish = spark.read.format("com.mongodb.spark.sql.DefaultSource")\
        .option("spark.mongodb.input.uri", "mongodb://localhost:27017/tfm_twitter.tweets_spanish").load()
    
    # nos quedamos solo con el atributo texto que es la información que nos interesa de cada dataset
    df_text_english = df_mongo_english[['text']]
    df_text_spanish = df_mongo_spanish[['text']]
    
    # pasamos a DF pandas
    df_pandas_english = df_text_english.toPandas()
    df_pandas_spanish = df_text_spanish.toPandas()
    
    return df_pandas_english, df_pandas_spanish


# Función de carga de datos desde los csv generados con anterioridad con registros ya con el sentimiento anotado
def carga_datos_csv(csv):
    # cargamos el csv
    if(csv=='english_full'):
        df_anotados = pd.read_csv('./data/df_result_english.csv', sep=',')
    elif(csv=='spanish_full'):
        df_anotados = pd.read_csv('./data/df_result_spanish.csv', sep=',')
    elif(csv=='english_neutro'):
        df_anotados = pd.read_csv('./data/df_result_english_neutral.csv', sep=',')
    elif(csv=='spanish_neutro'):
        df_anotados = pd.read_csv('./data/df_result_spanish_neutral.csv', sep=',')
    elif(csv=='english_noNeutro'):
        df_anotados = pd.read_csv('./data/df_result_english_noNeutral.csv', sep=',')
    elif(csv=='spanish_noNeutro'):
        df_anotados = pd.read_csv('./data/df_result_spanish_noNeutral.csv', sep=',')
        
    # eliminamos posibles valores de sentimiento vacio, pasándo el valor a -1 y eliminando el registro después
    df_anotados = df_anotados.fillna(-1)
    df_anotados = df_anotados[df_anotados['sentiment']!=-1]
    
    # el atributo sentimiento nos aseguramos que sea tipo int en lugar de float
    col_float = df_anotados.columns[df_anotados.dtypes == float]
    df_anotados[col_float] = df_anotados[col_float].astype(int)

    # como los datos en inglés del df full son muy grandes, para evitar problemas computacionales nos quedamos 
    # con una parte similar al tamaño de los datos que tenemos en español
    if(csv=='english_full'):
        df_anotados = df_anotados.sample(frac=0.04, replace=False, random_state=1)
    
    return df_anotados


# Función de visualización de DFs: número de registros y columnas, lista de columnas, recuento de los distintos
# valores de la columna sentimiento.
def visualizar_datos_df(df,tipo):
    print("num_rows: %d\tColumnas: %d\n" % (df.shape[0], df.shape[1]) )
    print("Columnas:\n", list(df.columns))
    print("\n")
    if(tipo==0):
        print("Recuento valores columna sentimiento:\n", pd.value_counts(df['sentiment']))
        print("\n")


# Función de limpieza de los tweets
def limpiar_tweet(tweet):
    # quitamos RT, @nombre_usuario, links y urls, hashtags, menciones, caracteres extraños o emoticonos
    tweet = re.sub('  +', ' ', tweet)
    # eliminar acentos
    tweet = ''.join((c for c in unicodedata.normalize('NFD', tweet) if unicodedata.category(c) != 'Mn'))  
    # convertir la repetición de una letra más de 2 veces a 1
    # biennnnn --> bien
    tweet = re.sub(r'(.)\1+', r'\1\1', tweet)
    # eliminar "RT", "@usuario", o los enlaces que es información que no sería útil analizar
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    tweet = re.sub('','',tweet).lower() 
    tweet = re.sub(r'http\S+', '', tweet) 
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    tweet = re.sub(r'#', '', tweet)
    tweet = re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet)
    tweet = re.sub(r'[0-9]', '', tweet) 

    return tweet 


# Función que le pasamos un Dataframe y nos lo devuelve con los datos preprocesados con la función anterior
def limpiar_df(df):
    df_clean = df.copy()
    df_clean['text'] = df_clean['text'].apply(limpiar_tweet)
    
    return df_clean


# Función que genera los conjuntos de train y test a partir de un dataframe. Reparto 90% train - 10% test
def split_df(df):
    X_eng = df['text']
    y_eng = df['sentiment']

    X, X_test, y, y_test = train_test_split(X_eng, y_eng, test_size=0.1, random_state=42)

    return X, X_test, y, y_test


# Función que genera el pipeline y parámetros del modelo, y lo entrena.
# Se centra la prueba en ver el resultado con distintos clasificadores para ver que tipo de algoritmo da los 
# mejores resultados. En cuanto al vectorizador se prueba tanto con CountVectorizer como con TDIDFVectorizer.
# No se definen muchos otros parámetros por cuestión de tiempo y del elevado coste computacional en el caso de
# jugar con muchos parámetros y varias posibilidades para cada uno.
def generar_modelo(clasificador, stopwords, X_train, y_train):
    # uso de tokenizer que es una buena práctica en el tratamiento de textos
    tokenizer = TweetTokenizer().tokenize

    pipeline = Pipeline([
        ('vectorizer', None),
        ('classifier', clasificador)]
    )
    
    # también se prueba a usar o no stopwords, que serían palabras muy comunes en el lenguaje, que no tienen valor
    # real a la hora de analizar el sentimiento de un texto y pueden no tenerse en cuenta.
    params = {
        'vectorizer': [CountVectorizer(binary=True,tokenizer=tokenizer),\
                       CountVectorizer(binary=False,tokenizer=tokenizer),\
                       TfidfVectorizer(use_idf=False, tokenizer=tokenizer),\
                       TfidfVectorizer(use_idf=True, tokenizer=tokenizer)],
        'vectorizer__stop_words': [None, stopwords],
    }

    # uso también de StratifiedKFold que es otra buena práctica
    skf = StratifiedKFold(n_splits=10, shuffle=True)

    modelo = GridSearchCV(pipeline, params, n_jobs=-1, cv=skf, refit='f1_weighted')
    modelo.fit(X_train, y_train)

    return modelo


# Función que evalua el modelo que le pasamos y saca las métricas que se quieren observar
def evaluar_modelos(modelo1, modelo2, x_eng_test, x_spa_test, y_eng_test, y_spa_test):
    # pintamos los mejores parámetros de cada modelo
    print("Mejores parámetros del modelo para datos en inglés: \n", modelo1.best_params_)
    print("\n")
    print("Mejores parámetros del modelo para datos en español: \n", modelo2.best_params_)

    # se usan los modelos entrenados para predecir los datos de test
    y_pred_eng = modelo1.predict(x_eng_test)
    y_pred_spa = modelo2.predict(x_spa_test)

    # pintamos las métricas de cada modelo
    print("\n")
    print("Métricas del modelo con datos en inglés:\n")
    print(classification_report(y_eng_test, y_pred_eng, target_names=None))
    print("\n")
    print("Métricas del modelo con datos en español:\n")
    print(classification_report(y_spa_test, y_pred_spa, target_names=None))

#### Carga de datos almacenados en MongoDB, de datos sin sentimiento calculado, por si probamos a utilizarlos con el modelo final.

In [3]:
df_mongo_english, df_mongo_spanish = carga_datos_mongo()

visualizar_datos_df(df_mongo_english,1)
visualizar_datos_df(df_mongo_spanish,1)

num_rows: 322364	Columnas: 1

Columnas:
 ['text']


num_rows: 79036	Columnas: 1

Columnas:
 ['text']




In [4]:
df_mongo_english.head(10)

,text
0,@EJFC26 Y Messi
1,RT @btsmoonchild64: These group photos deserve...
2,@rehankkhanNDS Overacting *
3,Pay day play day🤑🤑🤑🤑🤑
4,@pandaeyed1 Thank you!
5,RT @liamyoung: Strange that Tony Blair has sud...
6,Hard work puts you where good luck can find you.
7,RT @xCiphxr: When creative kids try playing co...
8,RT @bonang_m: I’m working on one as we speak. ...
9,"RT @akashbanerjee: After #PulwamaAttack, terro..."


In [5]:
df_mongo_spanish.head(10)

,text
0,@EJFC26 Y Messi
1,RT @MBelenAlegre: Hermoso y sensual viernes ❣️
2,RT @Foro_TV: Suman 47 muertos y 640 heridos po...
3,RT @revistaetcetera: .@lopezobrador_ dice que ...
4,"Me retracto de mi respuesta , él gobierno se s..."
5,Anda a saber si te esta eligiendo o sos lo úni...
6,RT @JCTrujilloCPCCS: Hay que propiciar una Con...
7,@RicciuP Terraplanistas
8,RT @oriolguellipuig: Jeje jeje https://t.co/NU...
9,quienes son las veganarcas? no quiero ser part...


#### Carga de datos desde csv generados, con el sentimiento de cada texto anotado, tanto con datos en inglés como en español, para utilizarlos en el entrenamiento y testeo de los modelos y ver el mejor clasificador.

In [6]:
df_csv_english = carga_datos_csv('english_full')
df_csv_spanish = carga_datos_csv('spanish_full')

visualizar_datos_df(df_csv_english,0)
visualizar_datos_df(df_csv_spanish,0)

num_rows: 70373	Columnas: 2

Columnas:
 ['text', 'sentiment']


Recuento valores columna sentimiento:
 2    35092
0    34714
1      567
Name: sentiment, dtype: int64


num_rows: 48658	Columnas: 2

Columnas:
 ['text', 'sentiment']


Recuento valores columna sentimiento:
 2    26204
0    19344
1     3110
Name: sentiment, dtype: int64




In [7]:
df_csv_english.head(10)

,text,sentiment
128038,@BradshawPhotogr seems i always end up at tx ...,2
491756,"@unitechy yeah, don't worry, you will!!! there...",0
470925,Excited that my email works reliably now with ...,0
491264,@MF213 yea that's the sad part,0
836490,@modeladrienne you let me know! I think you to...,0
371404,2155 - Well - Susan didn't make it ... Runne...,0
73350,..Before the cool runs out..Ima be trying my B...,2
1166160,@dbdc LMAO not today sir sorry sir I did go ye...,2
1070017,LOVES that lubbock is wet..its about time..no ...,2
229521,I want a golden retriever puppy!! soooo cute!!...,0


In [8]:
df_csv_spanish.head(10)

,text,sentiment
0,@PauladeLasHeras No te libraras de ayudar me/n...,1
1,@marodriguezb Gracias MAR,2
2,"Off pensando en el regalito Sinde, la que se v...",0
3,Conozco a alguien q es adicto al drama! Ja ja ...,2
4,Toca @crackoviadeTV3 . Grabación dl especial N...,2
5,Buen día todos! Lo primero mandar un abrazo gr...,2
6,Desde el escaño. Todo listo para empezar #endi...,2
7,Bdías. EM no se ira de puente. Si vosotros os ...,2
8,Un sistema económico q recorta dinero para pre...,2
9,#programascambiados caca d ajuste,0


#### Realizamos una limpieza de los textos para eliminar caracteres que no tengan valor para el análisis a realizar.

In [9]:
df_clean_english = limpiar_df(df_csv_english)
df_clean_spanish = limpiar_df(df_csv_spanish)

visualizar_datos_df(df_clean_english,0)
visualizar_datos_df(df_clean_spanish,0)

num_rows: 70373	Columnas: 2

Columnas:
 ['text', 'sentiment']


Recuento valores columna sentimiento:
 2    35092
0    34714
1      567
Name: sentiment, dtype: int64


num_rows: 48658	Columnas: 2

Columnas:
 ['text', 'sentiment']


Recuento valores columna sentimiento:
 2    26204
0    19344
1     3110
Name: sentiment, dtype: int64




In [10]:
df_clean_english.head(10)

,text,sentiment
128038,seems i always end up at tx schl email lori...,2
491756,yeah don t worry you will there s still ...,0
470925,excited that my email works reliably now with ...,0
491264,yea that s the sad part,0
836490,you let me know i think you too busy for li...,0
371404,well susan didn t make it runner up to...,0
73350,before the cool runs out ima be trying my b...,2
1166160,lmao not today sir sorry sir i did go yester...,2
1070017,loves that lubbock is wet its about time no ...,2
229521,i want a golden retriever puppy soo cute d...,0


In [11]:
df_clean_spanish.head(10)

,text,sentiment
0,no te libraras de ayudar me nos besos y gra...,1
1,gracias mar,2
2,off pensando en el regalito sinde la que se v...,0
3,conozco a alguien q es adicto al drama ja ja ...,2
4,toca grabacion dl especial navideno mari ...,2
5,buen dia todos lo primero mandar un abrazo gr...,2
6,desde el escano todo listo para empezar endia...,2
7,bdias em no se ira de puente si vosotros os ...,2
8,un sistema economico q recorta dinero para pre...,2
9,programascambiados caca d ajuste,0


#### Separamos los datos tanto en inglés como en español en conjuntos de entrenamiento y de testing.

In [12]:
X_eng_train, X_eng_test, y_eng_train, y_eng_test = split_df(df_clean_english)
X_spa_train, X_spa_test, y_spa_train, y_spa_test = split_df(df_clean_spanish)

#### Prueba modelo final.
A la hora de aplicar machine learning para abordar problemas como éste del análisis de sentimiento, hay dos tipos de aprendizaje: supervisado y no supervisado.
En este trabajo optamos por aplicar aprendizaje supervisado, que son métodos basados en algoritmos de aprendizaje automático que necesitan para entrenarse un conjunto de datos ya etiquetados.

Vamos a probar con el modelo final y sus parámetros que probamos anteriormente, con 4 algoritmos de clasificación distintos, usando los siguientes: Naive Bayes, Máquinas de vector soporte, K vecinos más cercanos, y árboles de decisión.

In [9]:
# cargamos las stopwords para cada idioma
spanish_stopwords = stopwords.words('spanish')
english_stopwords = stopwords.words('english')

#### Algoritmo Naive Bayes

In [15]:
# generamos los modelos para datos en inglés y en español
clasificador = MultinomialNB()

model_eng_naive = generar_modelo(clasificador, english_stopwords, X_eng_train, y_eng_train)
model_spa_naive = generar_modelo(clasificador, spanish_stopwords, X_spa_train, y_spa_train)

In [16]:
# evaluamos los modelos
evaluar_modelos(model_eng_naive, model_spa_naive, X_eng_test, X_spa_test, y_eng_test, y_spa_test)

Mejores parámetros del modelo para datos en inglés: 
 {'vectorizer': TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<bound method TweetTokenizer.tokenize of <nltk.tokenize.casual.TweetTokenizer object at 0x1a1ebf82b0>>,
        use_idf=False, vocabulary=None), 'vectorizer__stop_words': None}


Mejores parámetros del modelo para datos en español: 
 {'vectorizer': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
 

/Users/josemanuel/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


#### Algoritmo K-vecinos

In [14]:
# generamos los modelos para datos en inglés y en español
clasificador = KNeighborsClassifier()

model_eng_vecinos = generar_modelo(clasificador, english_stopwords, X_eng_train, y_eng_train)
model_spa_vecinos = generar_modelo(clasificador, spanish_stopwords, X_spa_train, y_spa_train)

In [15]:
# evaluamos los modelos
evaluar_modelos(model_eng_vecinos, model_spa_vecinos, X_eng_test, X_spa_test, y_eng_test, y_spa_test)

Mejores parámetros del modelo para datos en inglés: 
 {'vectorizer': CountVectorizer(analyzer='word', binary=True, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<bound method TweetTokenizer.tokenize of <nltk.tokenize.casual.TweetTokenizer object at 0x1a2471b160>>,
        vocabulary=None), 'vectorizer__stop_words': None}


Mejores parámetros del modelo para datos en español: 
 {'vectorizer': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        toke

#### Algoritmo Máquinas vector soporte

In [16]:
# generamos los modelos para datos en inglés y en español
clasificador = LinearSVC()

model_eng_svms = generar_modelo(clasificador, english_stopwords, X_eng_train, y_eng_train)
model_spa_svms = generar_modelo(clasificador, spanish_stopwords, X_spa_train, y_spa_train)

In [17]:
# evaluamos los modelos
evaluar_modelos(model_eng_svms, model_spa_svms, X_eng_test, X_spa_test, y_eng_test, y_spa_test)

Mejores parámetros del modelo para datos en inglés: 
 {'vectorizer': TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<bound method TweetTokenizer.tokenize of <nltk.tokenize.casual.TweetTokenizer object at 0x1a28d09710>>,
        use_idf=False, vocabulary=None), 'vectorizer__stop_words': None}


Mejores parámetros del modelo para datos en español: 
 {'vectorizer': TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smoot

#### Algoritmo árboles de decisión

In [18]:
# generamos los modelos para datos en inglés y en español
clasificador = DecisionTreeClassifier()

model_eng_tree = generar_modelo(clasificador, english_stopwords, X_eng_train, y_eng_train)
model_spa_tree = generar_modelo(clasificador, spanish_stopwords, X_spa_train, y_spa_train)

In [19]:
# evaluamos los modelos
evaluar_modelos(model_eng_tree, model_spa_tree, X_eng_test, X_spa_test, y_eng_test, y_spa_test)

Mejores parámetros del modelo para datos en inglés: 
 {'vectorizer': CountVectorizer(analyzer='word', binary=True, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<bound method TweetTokenizer.tokenize of <nltk.tokenize.casual.TweetTokenizer object at 0x1a2531f860>>,
        vocabulary=None), 'vectorizer__stop_words': None}


Mejores parámetros del modelo para datos en español: 
 {'vectorizer': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None,
        stop_words=['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se', 'las', 'por

#### Resultados del modelo tras probarlo con los 4 algoritmos de clasificación.

Los mejores resultados vuelven a obtenerse de forma general, teniendo en cuenta las clases en conjunto y las distintas métricas que se obtienen en cada evaluación, con las máquinas vector soporte, en este caso el algoritmo LinearSVC.

Los resultados suelen rondar aproximadamente sobre el 75% con los datos en Inglés, y el 80% con los datos en Español, llegando a sus máximos valores con LinearSVC como hemos dicho, donde se alcanza un 77% y un 82% respectivamente.

Como ya se dijo anteriormente en el trabajo, se podrían probar muchos parámetros más, con distintos valores y combinaciones, para optimizar el modelo y poder subir los % para cada algoritmo, pero requiere un tiempo y un coste computacional que no se puede asumir.

Nos quedamos con el clasificador LinearSVC y un modelo como el probado aquí, para aplicarlo en streaming a los tweets que se vayan recogiendo.

#### Vamos a cargar los csv que se generaron en inglés y español con el mismo número de registros positivos y negativos en cada dataset, y los que tienen el mismo número de registros positivos, negativos y neutros por dataset.

Probaremos el modelo anterior con el clasificador que mejor resultado se ha obtenido con estos datasets, para ver como influye el reparto de clases por dataset, y el número de registros, además de si hay 2 o 3 clases.

In [10]:
# cargamos los 4 datasets
df_csv_english_neutro = carga_datos_csv('english_neutro')
df_csv_spanish_neutro = carga_datos_csv('spanish_neutro')
df_csv_english_noNeutro = carga_datos_csv('english_noNeutro')
df_csv_spanish_noNeutro = carga_datos_csv('spanish_noNeutro')

visualizar_datos_df(df_csv_english_neutro,0)
visualizar_datos_df(df_csv_spanish_neutro,0)
visualizar_datos_df(df_csv_english_noNeutro,0)
visualizar_datos_df(df_csv_spanish_noNeutro,0)

num_rows: 42000	Columnas: 2

Columnas:
 ['text', 'sentiment']


Recuento valores columna sentimiento:
 2    14000
1    14000
0    14000
Name: sentiment, dtype: int64


num_rows: 9000	Columnas: 2

Columnas:
 ['text', 'sentiment']


Recuento valores columna sentimiento:
 2    3000
1    3000
0    3000
Name: sentiment, dtype: int64


num_rows: 80000	Columnas: 2

Columnas:
 ['text', 'sentiment']


Recuento valores columna sentimiento:
 2    40000
0    40000
Name: sentiment, dtype: int64


num_rows: 38000	Columnas: 2

Columnas:
 ['text', 'sentiment']


Recuento valores columna sentimiento:
 2    19000
0    19000
Name: sentiment, dtype: int64




In [11]:
# limpiamos los datos
df_clean_english_neutro = limpiar_df(df_csv_english_neutro)
df_clean_spanish_neutro = limpiar_df(df_csv_spanish_neutro)
df_clean_english_noNeutro = limpiar_df(df_csv_english_noNeutro)
df_clean_spanish_noNeutro = limpiar_df(df_csv_spanish_noNeutro)

In [12]:
# hacemos el reparto de datos para entrenamiento y testing
X_eng_train_neu, X_eng_test_neu, y_eng_train_neu, y_eng_test_neu = split_df(df_clean_english_neutro)
X_spa_train_neu, X_spa_test_neu, y_spa_train_neu, y_spa_test_neu = split_df(df_clean_spanish_neutro)
X_eng_train_noNeu, X_eng_test_noNeu, y_eng_train_noNeu, y_eng_test_noNeu = split_df(df_clean_english_noNeutro)
X_spa_train_noNeu, X_spa_test_noNeu, y_spa_train_noNeu, y_spa_test_noNeu = split_df(df_clean_spanish_noNeutro)

In [13]:
# probamos el modelo con los 4 datasets anteriores
clasificador = LinearSVC()

model_eng_svms_neu = generar_modelo(clasificador, english_stopwords, X_eng_train_neu, y_eng_train_neu)
model_spa_svms_neu = generar_modelo(clasificador, spanish_stopwords, X_spa_train_neu, y_spa_train_neu)
model_eng_svms_noNeu = generar_modelo(clasificador, english_stopwords, X_eng_train_noNeu, y_eng_train_noNeu)
model_spa_svms_noNeu = generar_modelo(clasificador, spanish_stopwords, X_spa_train_noNeu, y_spa_train_noNeu)

In [14]:
# evaluamos los modelos con 3 clases: positivos, negativos y neutros
evaluar_modelos(model_eng_svms_neu, model_spa_svms_neu, X_eng_test_neu, X_spa_test_neu,\
                y_eng_test_neu, y_spa_test_neu)

Mejores parámetros del modelo para datos en inglés: 
 {'vectorizer': TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<bound method TweetTokenizer.tokenize of <nltk.tokenize.casual.TweetTokenizer object at 0x1a3a23f470>>,
        use_idf=True, vocabulary=None), 'vectorizer__stop_words': None}


Mejores parámetros del modelo para datos en español: 
 {'vectorizer': TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth

In [15]:
# evaluamos los modelos con 2 clases: positivos y negativos
evaluar_modelos(model_eng_svms_noNeu, model_spa_svms_noNeu, X_eng_test_noNeu, X_spa_test_noNeu,\
                y_eng_test_noNeu, y_spa_test_noNeu)

Mejores parámetros del modelo para datos en inglés: 
 {'vectorizer': TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<bound method TweetTokenizer.tokenize of <nltk.tokenize.casual.TweetTokenizer object at 0x1a3a23f7b8>>,
        use_idf=False, vocabulary=None), 'vectorizer__stop_words': None}


Mejores parámetros del modelo para datos en español: 
 {'vectorizer': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
 

Se obtienen los siguientes resultados:
- Modelo entrenado y testeado con los datasets con 3 clases y equilibrados en el número de registros por clase: 66% con datos en inglés, y 77% con datos en español.
- Modelo entrenado y testeado con los datasets con 2 clases (positivo y negativo) y equilibrados en el número de registros por clase: 78% y 94%, para inglés y español respectivamente.

Como era de suponer se obtienen mejores resultados sólo con 2 clases, y se sigue manteniendo la tendencia de mejor clasificación con datos en español que en inglés. De todos modos también hay que pensar que los datasets son más pequeños y con menor número de datos los resultados pueden ser menos fiables.
Además la realidad es que al menos deben tener en cuenta 3 clases, ya que como mínimo siempre se deben clasificar los sentimientos en negativo, neutro o positivo.

Se puede apreciar como entrenando con datasets equilibrados entre sus distintas clases, se obtienen resultados buenos y similares para cada clase, no como ocurría al entrenar y testear con un dataset desequilibrado, que la clase con menos registros obtiene unos pobres resultados.

#### Prueba a predecir el sentimiento de los datos cargados desde MongoDB con el modelo con mejores resultados.

In [20]:
# datos cargados desde MongoDB
# limpiamos los textos
df_clean_mongo_english = limpiar_df(df_mongo_english)
df_clean_mongo_spanish = limpiar_df(df_mongo_spanish)

In [21]:
df_clean_mongo_english.head(10)

,text
0,y messi
1,these group photos deserve more attention
2,overacting
3,pay day play day
4,thank you
5,strange that tony blair has suddenly become...
6,hard work puts you where good luck can find you
7,when creative kids try playing comp
8,m i m working on one as we speak thank you...
9,after pulwamaattack terrorists scored a bi...


In [23]:
df_clean_mongo_spanish.head(10)

,text
0,y messi
1,hermoso y sensual viernes
2,tv suman muertos y heridos por explosion ...
3,dice que fue tan grande la gloria de be...
4,me retracto de mi respuesta el gobierno se s...
5,anda a saber si te esta eligiendo o sos lo uni...
6,hay que propiciar una consulta popular para...
7,terraplanistas
8,jeje jeje
9,quienes son las veganarcas no quiero ser part...


In [24]:
# Predecimos usando el modelo de SVMs que ha obtenido los mejores resultados
y_pred_mongo_eng = model_eng_svms.predict(df_clean_mongo_english.text)
y_pred_mongo_spa = model_spa_svms.predict(df_clean_mongo_spanish.text)

In [25]:
# hacemos una copia de los dataframes con los textos limpios, y les añadimos la columna sentimiento con los valores
# que se han predecido con el modelo.
df_predict_english = df_clean_mongo_english.copy()
df_predict_spanish = df_clean_mongo_spanish.copy()

df_predict_english['sentiment'] = y_pred_mongo_eng
df_predict_spanish['sentiment'] = y_pred_mongo_spa

In [26]:
df_predict_english.head(30)

,text,sentiment
0,y messi,2
1,these group photos deserve more attention,2
2,overacting,2
3,pay day play day,2
4,thank you,2
5,strange that tony blair has suddenly become...,0
6,hard work puts you where good luck can find you,2
7,when creative kids try playing comp,2
8,m i m working on one as we speak thank you...,2
9,after pulwamaattack terrorists scored a bi...,2


In [27]:
df_predict_spanish.head(30)

,text,sentiment
0,y messi,2
1,hermoso y sensual viernes,2
2,tv suman muertos y heridos por explosion ...,0
3,dice que fue tan grande la gloria de be...,0
4,me retracto de mi respuesta el gobierno se s...,0
5,anda a saber si te esta eligiendo o sos lo uni...,0
6,hay que propiciar una consulta popular para...,2
7,terraplanistas,2
8,jeje jeje,2
9,quienes son las veganarcas no quiero ser part...,0


Se observan los primeros 30 tweets con el sentimiento predecido por el modelo, y se pueden ver varios ejemplos de un funcionamiento aceptablemente correcto.

También se puede ver como se predicen más etiquetas con valor positivo o 2, que del resto. Esto es debido a que si se ve al inicio el reparto de valores en los datasets usados para entrenar los modelos, el valor positivo es el que más número de datos tiene. Además, no se ve ningún valor neutral, ya que los datasets tienen muy pocos valores neutrales, y por tanto el modelo no va a predecir fácilmente esta etiqueta.

Para obtener un modelo con el mejor funcionamiento posible, haría falta que estuviera totalmente balanceado entre todas sus clases.